![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [ ]:
 # Start your code here!
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from data import CustomerAnalytics
from data import BuildModel
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from data import BuildModel

# Predicting that a student will not what to change jobs

In [ ]:
model = BuildModel()

In [ ]:
# my_dict = {'gender': ['Other', 'Male', 'Female'],
#  'relevant_experience': ['Has relevant experience', 'No relevant experience'],
#  'enrolled_university': ['Full time course', 'Part time course', 'no_enrollment'],
#  'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
#  'major_discipline': ['Arts', 'Business Degree', 'Humanities', 'No Major', 'Other', 'STEM'],
#  'experience': ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20'],
#  'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
#  'company_type': ['Early Stage Startup', 'Funded Startup', 'NGO', 'Other', 'Public Sector', 'Pvt Ltd'],
#  'last_new_job': ['never', '1', '2', '3', '4', '>4'],
#  'training_hours': 'Any value'}

# my_dict

In [ ]:
model.dump()

In [ ]:
model.evaluate()
model.